In [60]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel
from scipy.sparse import csr_matrix
from sklearn.metrics import accuracy_score
import torch
import os
import seaborn as sns
from torch.utils.data import DataLoader,TensorDataset
import pickle
import json
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score
from sae.preprocess import get_meta_info,get_freq_emmbeddings, get_tf_idfs_emmbeddings,get_bert_embeddings
from sae.models import DEEPmodel_2
import time
import joblib
%autoreload 2

In [62]:
data_valid = pd.read_csv('validation_data.csv')
data_valid.rename(columns={'revue/score':'rating',"revue/résumé":"summary","revue/texte":"comment"}, inplace=True)
data_valid["summary"] = np.where(pd.isnull(data_valid["summary"]), '', data_valid["summary"])
data_valid["comment"] = np.where(pd.isnull(data_valid["comment"]), '', data_valid["comment"])
len(data_valid)

2000

# Cacul des metas attribut

In [63]:
#on recharge tous les fichiers json pour verifier
with open('titre_info.json', 'r') as fp:
    titre_info = json.load(fp)
with open('author_data.json', 'r') as fp:
    author_data = json.load(fp)
with open('genre_data.json', 'r') as fp:
    genre_data = json.load(fp)
with open('editor_data.json', 'r') as fp:
    editor_data = json.load(fp)


In [64]:

ratings_author, nb_rat_auhtor, ratings_genre, nb_rat_genre, ratings_editor, nb_rat_editor =  get_meta_info(data_valid,titre_info, author_data,genre_data,editor_data)

In [65]:
#on rajoute les colonnes dans le dataframe ratings
data_valid['auteur_note'] = ratings_author
data_valid['auteur_nb'] = nb_rat_auhtor
data_valid['genre_note'] = ratings_genre
data_valid['genre_nb'] = nb_rat_genre
data_valid['editor_note'] = ratings_editor
data_valid['editor_nb'] = nb_rat_editor


In [66]:
X_meta = data_valid[['auteur_note','auteur_nb','genre_note','genre_nb','editor_note','editor_nb']].values

X_meta.shape


(2000, 6)

# Calcul des embeddings

In [67]:

#calcule les embeddings ou utilise les embeddings pre-entrainés
mode = "summary"
#on remplace les nan par des chaines de caractères vides

# Load the saved model
with open('model_freq_emb.pkl', 'rb') as file:
    model_freq = pickle.load(file)
X_emb_freq,_ = get_freq_emmbeddings(data_valid[mode].values,model_freq)

In [ ]:

Y_valid_rating = data_valid['rating'].values.astype(int)
Y_valid_sentiment = data_valid['rating'].values.astype(int)
Y_valid_sentiment[Y_valid_sentiment < 3] = 0
Y_valid_sentiment[Y_valid_sentiment == 3] = 1
Y_valid_sentiment[Y_valid_sentiment > 3] = 2


# Embeding

## mots simple

In [69]:
Freqmodel = joblib.load('freq_model.pkl')

In [70]:
X_Freq,inutile = get_freq_emmbeddings(list(data_valid[mode].values),model=Freqmodel)
X_valid_Freq = np.concatenate((X_meta,X_Freq),axis=1)
X_Freq

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### SKlearn

In [71]:
nb_model = joblib.load('freq_gradientboosting.pkl')
result=nb_model.predict(X_valid_Freq)
print(result)
print(Y_valid_sentiment)
accuracy_score(result,Y_valid_sentiment)

[2 2 2 ... 2 0 2]
[2 2 2 ... 0 0 0]


0.8005

### deep

In [72]:
X_valid_Freq = np.concatenate((X_meta,X_Freq),axis=1)
batch_size = 128
valid_dataset_Freq_sent = TensorDataset(torch.tensor(X_valid_Freq),torch.tensor(Y_valid_sentiment, dtype=torch.long))
valid_loader_Freq_sent = DataLoader(valid_dataset_Freq_sent, batch_size=batch_size, shuffle=False)

In [73]:
modeldSimple = DEEPmodel_2(input_size=1006, features = [512,512,512,512,256], sub_features =[256,256], nb_class=3)
modeldSimple.load_state_dict(torch.load('Deep_freq.pth'))

<All keys matched successfully>

In [74]:
modeldSimple.eval()
with torch.no_grad():
    y_pred = []
    for i, (x, y) in enumerate(valid_loader_Freq_sent):
        x = x.float()  
        y_pred_batch = modeldSimple(x).argmax(1)
        y_pred.append(y_pred_batch)  # Move to CPU for evaluation
    
    y_pred = torch.cat(y_pred)
acc_valid = accuracy_score(Y_valid_sentiment, y_pred.numpy())
print("accuracyh valid :", acc_valid)

accuracyh valid : 0.8395


## tf idf

In [75]:
idfmodel = joblib.load('tfidf_model.pkl')

In [76]:
X_idf,inutile = get_tf_idfs_emmbeddings(list(data_valid[mode].values),model=idfmodel)
X_valid_idf = np.concatenate((X_meta,X_idf),axis=1)
X_idf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Sklearn

In [77]:
# Charger le modèle
nb_model = joblib.load('idf_gradientboosting.pkl')
result=nb_model.predict(X_valid_idf)
print(result)
print(Y_valid_sentiment)
accuracy_score(result,Y_valid_sentiment)

[2 2 2 ... 2 0 2]
[2 2 2 ... 0 0 0]


0.801

### deep

In [78]:
batch_size = 128
valid_dataset_idf_sent = TensorDataset(torch.tensor(X_valid_idf),torch.tensor(Y_valid_sentiment, dtype=torch.long))
valid_loader_idf_sent = DataLoader(valid_dataset_idf_sent, batch_size=batch_size, shuffle=False)

In [79]:
modeldidf = DEEPmodel_2(input_size=1006, features = [512,512,512,512,256], sub_features =[256,256], nb_class=3)
modeldidf.load_state_dict(torch.load('Deep_idf.pth'))

<All keys matched successfully>

In [80]:
modeldidf.eval()
with torch.no_grad():
    y_pred = []
    for i, (x, y) in enumerate(valid_loader_idf_sent):
        x = x.float()  
        y_pred_batch = modeldidf(x).argmax(1)
        y_pred.append(y_pred_batch)  # Move to CPU for evaluation
    
    y_pred = torch.cat(y_pred)
acc_valid = accuracy_score(Y_valid_sentiment, y_pred.numpy())
print("accuracyh valid :", acc_valid)

accuracyh valid : 0.837


## bert

In [81]:
X_bert = get_bert_embeddings(list(data_valid[mode].values)).numpy()
X_valid_bert = np.concatenate((X_meta,X_bert),axis=1)

****************


### SKlearn

In [82]:
# Charger le modèle
nb_model = joblib.load('bert_gradientboosting.pkl')
result=nb_model.predict(X_valid_bert)
print(result)
print(Y_valid_sentiment)
accuracy_score(result,Y_valid_sentiment)


[2 2 2 ... 0 0 2]
[2 2 2 ... 0 0 0]


0.8265

### deep

In [83]:

batch_size = 128
valid_dataset_bert_sent = TensorDataset(torch.tensor(X_valid_bert),torch.tensor(Y_valid_sentiment, dtype=torch.long))
valid_loader_bert_sent = DataLoader(valid_dataset_bert_sent, batch_size=batch_size, shuffle=False)

In [84]:
model = DEEPmodel_2(input_size=774, features = [512,512,512,512,256], sub_features =[256,256], nb_class=3)
model.load_state_dict(torch.load('Deep_bert.pth'))

<All keys matched successfully>

In [85]:
model.eval()
with torch.no_grad():
    y_pred = []
    for i, (x, y) in enumerate(valid_loader_bert_sent):
        x = x.float()  
        y_pred_batch = model(x).argmax(1)
        y_pred.append(y_pred_batch)  # Move to CPU for evaluation
    
    y_pred = torch.cat(y_pred)
acc_valid = accuracy_score(Y_valid_sentiment, y_pred.numpy())
print("accuracyh valid :", acc_valid)

accuracyh valid : 0.897
